In [1]:
import os
import csv

import cv2
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf

## Model definition

In [2]:
### Model Parameters
conv_dropout = 0.4
dense_dropout = 0.5
l2_regularization = 0.001

# l1_regularizer = tf.keras.regularizers.l1(0.01)
l2_regularizer = None #tf.keras.regularizers.l2(l2_regularization)

CROP_UP = 50
CROP_DOWN = 30
final_height = 160 - (CROP_UP + CROP_DOWN)

In [3]:
### Aux functions for batchnorm layers
def conv2d_batch_norm(*args, **kwargs):

    activation = kwargs.pop("activation", None)
    batch_norm = kwargs.pop("batch_norm", {})

    net = tf.layers.conv2d(*args, **kwargs)
    net = tf.layers.batch_normalization(net, **batch_norm)

    return activation(net) if activation else net

def dense_batch_norm(*args, **kwargs):

    activation = kwargs.pop("activation", None)
    batch_norm = kwargs.pop("batch_norm", {})

    net = tf.layers.dense(*args, **kwargs)
    net = tf.layers.batch_normalization(net, **batch_norm)

    return activation(net) if activation else net

In [4]:
mu = 0
sigma = 0.1
init = None #tf.initializers.truncated_normal(mean = mu, stddev = sigma)


def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features, [-1, final_height, 320, 3])

    general_ops = dict(
#         activation = activation,
        batch_norm = dict(training = mode == tf.estimator.ModeKeys.TRAIN)
#         kernel_regularizer = tf.contrib.layers.l2_regularizer(l2_regularization),
    )
                                             
    input_normalized = tf.layers.batch_normalization(input_layer, training = mode == tf.estimator.ModeKeys.TRAIN)
    
    # Convolutional Layer #1
    conv1 = conv2d_batch_norm(
      inputs=input_normalized,
      filters=32,
      kernel_size=[5, 5],
      padding="valid",
      activation=tf.nn.relu, kernel_initializer=init,
      **general_ops)
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    pool1 = tf.layers.dropout(
        inputs=pool1, 
        rate=conv_dropout, 
        training= mode == tf.estimator.ModeKeys.TRAIN)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = conv2d_batch_norm(
      inputs=pool1,
      filters=32,
      kernel_size=[3, 3],
      padding="valid",
      activation=tf.nn.relu, kernel_initializer=init,
    **general_ops)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    pool2 = tf.layers.dropout(
        inputs=pool2, 
        rate=conv_dropout, 
        training= mode == tf.estimator.ModeKeys.TRAIN)
                                             
    # Convolutional Layer #3 and Pooling Layer #3
    conv3 = conv2d_batch_norm(
      inputs=pool2,
      filters=64,
      kernel_size=[3, 3],
      padding="valid",
      activation=tf.nn.relu, kernel_initializer=init,
    **general_ops)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)
    pool3 = tf.layers.dropout(
        inputs=pool3, 
        rate=conv_dropout, 
        training= mode == tf.estimator.ModeKeys.TRAIN)  
                                             
    # Convolutional Layer #4 and Pooling Layer #4
    conv4 = conv2d_batch_norm(
      inputs=pool3,
      filters=64,
      kernel_size=[3, 3],
      padding="valid",
      activation=tf.nn.relu, kernel_initializer=init,
    **general_ops)
    pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)
    pool4 = tf.layers.dropout(
        inputs=pool4, 
        rate=conv_dropout, 
        training= mode == tf.estimator.ModeKeys.TRAIN)  
                                             
    # Dense Layers
    pool_flat = tf.layers.flatten(conv4)
    
    dense1 = dense_batch_norm(inputs=pool_flat,
                             units=1024,
                             activation=tf.nn.relu,
                             kernel_initializer=init,
                             **general_ops)
    dense1 = tf.layers.dropout(
        inputs=dense1, 
        rate=dense_dropout, 
        training= mode == tf.estimator.ModeKeys.TRAIN)
    
    dense2 = dense_batch_norm(inputs=dense1,
                             units=512,
                             activation=tf.nn.relu,
                             kernel_initializer=init,
                             **general_ops)
    dense2 = tf.layers.dropout(
        inputs=dense2, 
        rate=dense_dropout, 
        training= mode == tf.estimator.ModeKeys.TRAIN)
                                             
    dense3 = dense_batch_norm(inputs=dense2,
                             units=128,
                             activation=tf.nn.relu,
                             kernel_initializer=init,
                             **general_ops)
    dense3 = tf.layers.dropout(
        inputs=dense3, 
        rate=dense_dropout, 
        training= mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    preds = tf.layers.dense(inputs=dense3, units=1, kernel_initializer=init, use_bias=False)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "steering": preds,
    "image": input_layer
    }


    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    with tf.name_scope('loss'):
            loss = tf.losses.mean_squared_error(
                labels=labels, predictions=tf.squeeze(preds, axis = 1), scope='loss')
            tf.summary.scalar('loss', loss)
    
    # Accuracy    
    with tf.name_scope('mae'):
            mae = tf.metrics.mean_absolute_error(
                labels=labels, predictions=tf.squeeze(preds, axis = 1), name='mae')
            tf.summary.scalar('mae', mae[1])

    # Create a hook to print acc, loss & global step every 100 iter.   
    train_hook_list= []
    train_tensors_log = {'mae': mae[1],
                         'loss': loss}
    train_hook_list.append(tf.train.LoggingTensorHook(
        tensors=train_tensors_log, every_n_iter=100))

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:

        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
#             optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
#             optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)
            train_op = optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())
    
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            loss=loss, 
            train_op=train_op, 
            training_hooks=train_hook_list)

    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops={'mae/mae': mae}, evaluation_hooks=None)


In [5]:
# Create the Estimator
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/pilotnetV1")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_device_fn': None, '_protocol': None, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_task_id': 0, '_service': None, '_global_id_in_cluster': 0, '_train_distribute': None, '_save_summary_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_model_dir': '/tmp/pilotnetV1', '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d049e3b00>, '_eval_distribute': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_task_type': 'worker'}


## Dataset creation

In [6]:
### folder where the data is
DATASET = "data_mine"

In [7]:
### Load csv as a pandas dataframe
header = ["center", "left", "right", "steering", "throttle", "break", "speed"]
df = pd.read_csv(os.path.join(DATASET, "driving_log.csv"), header=None, names=header)
df.head()

,center,left,right,steering,throttle,break,speed
0,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000020
1,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000011
2,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000011
3,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000021
4,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,/home/charlie/data/charlie/self-driving-nano/C...,0.0,0.0,0,0.000016


In [8]:
def get_image_path(filename):
    return os.path.join(DATASET,"IMG", filename.split("/")[-1])

In [9]:
### Make paths relative to this jupyter notebook
df["center"] = df.center.apply(get_image_path)
df["left"] = df.left.apply(get_image_path)
df["right"] = df.right.apply(get_image_path)
df.head()

,center,left,right,steering,throttle,break,speed
0,data_mine/IMG/center_2019_02_03_21_24_02_332.jpg,data_mine/IMG/left_2019_02_03_21_24_02_332.jpg,data_mine/IMG/right_2019_02_03_21_24_02_332.jpg,0.0,0.0,0,0.000020
1,data_mine/IMG/center_2019_02_03_21_24_02_398.jpg,data_mine/IMG/left_2019_02_03_21_24_02_398.jpg,data_mine/IMG/right_2019_02_03_21_24_02_398.jpg,0.0,0.0,0,0.000011
2,data_mine/IMG/center_2019_02_03_21_24_02_467.jpg,data_mine/IMG/left_2019_02_03_21_24_02_467.jpg,data_mine/IMG/right_2019_02_03_21_24_02_467.jpg,0.0,0.0,0,0.000011
3,data_mine/IMG/center_2019_02_03_21_24_02_536.jpg,data_mine/IMG/left_2019_02_03_21_24_02_536.jpg,data_mine/IMG/right_2019_02_03_21_24_02_536.jpg,0.0,0.0,0,0.000021
4,data_mine/IMG/center_2019_02_03_21_24_02_605.jpg,data_mine/IMG/left_2019_02_03_21_24_02_605.jpg,data_mine/IMG/right_2019_02_03_21_24_02_605.jpg,0.0,0.0,0,0.000016


In [10]:
### Split train and dev set 
train_samples, validation_samples = train_test_split(df, test_size=0.2)

In [11]:
# Get filenames and labels (steerings)
c_filenames = train_samples.center.values
c_labels = train_samples.steering.values

val_filenames = validation_samples.center.values
val_labels = validation_samples.steering.values

In [12]:
# Add side cameras to train set
steer_correction = 0.2

l_filenames = train_samples.left.values
l_labels = train_samples.steering.values + steer_correction

r_filenames = train_samples.right.values
r_labels = train_samples.steering.values - steer_correction

filenames = np.concatenate([c_filenames, l_filenames, r_filenames])
labels = np.concatenate([c_labels, l_labels, r_labels])

In [13]:
# Aux function to crop the images
def get_crop_window(crop_up, crop_down):
    final_height = 160 - (crop_up + crop_down)
    final_width = 320

    return [
        crop_up,
        0,
        final_height,
        final_width,
    ]

In [14]:
# Function that returns the image decoded from jpg and label

def parse_function(filename, label):
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)
    
    image = tf.image.decode_and_crop_jpeg(
            image_string,
            crop_window = get_crop_window(CROP_UP, CROP_DOWN),
            channels = 3
        )

    # This will convert to float values in [0, 1]
#     image = tf.image.convert_image_dtype(image, tf.float32)

#     image = tf.image.resize_images(image, [160, 320])
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.float32)
    return image, label

### Use of TF DataSet API

In [15]:
# Define parameters
BATCH_SIZE = 32
EPOCHS = 25
steps_per_epoch = len(train_samples)*3//BATCH_SIZE

In [16]:
# Create training dataset
def create_train_dataset():
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.shuffle(len(filenames))
    dataset = dataset.map(parse_function, num_parallel_calls=4)
    dataset = dataset.repeat(EPOCHS)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(1)
    return dataset

In [17]:
# Create dev dataset
def create_eval_dataset():
    val_dataset = tf.data.Dataset.from_tensor_slices((val_filenames, val_labels))
    val_dataset = val_dataset.map(parse_function, num_parallel_calls=4)
    val_dataset = val_dataset.repeat(EPOCHS)
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.prefetch(1)
    return val_dataset

In [18]:
# # Get image example for trainig
# sess = tf.Session()
# value = dataset.make_one_shot_iterator().get_next()
# x,y = sess.run(value)

In [19]:
# print(x.shape, y[0])
# plt.imshow(x[0])

## Training

In [20]:
def serving_input_fn():

    input_image = tf.placeholder(
        dtype=tf.float32,
        shape=[None, None, None, 3],
        name="input_image"
    )

#     images = tf.image.resize_images(input_image, [params.image_height, params.image_width])
    images = input_image
    images = tf.image.crop_to_bounding_box(images, *get_crop_window(CROP_UP, CROP_DOWN))

    images = tf.cast(images, tf.float32)

    return tf.estimator.export.TensorServingInputReceiver(
        features = images,
        receiver_tensors = input_image
    )

In [21]:
exporter = tf.estimator.LatestExporter(
    "test_exporter",
    lambda: serving_input_fn(),
)

In [22]:
# Specs
train_spec = tf.estimator.TrainSpec(
    input_fn=create_train_dataset,
    max_steps=EPOCHS*steps_per_epoch)

eval_spec = tf.estimator.EvalSpec(
    input_fn=create_eval_dataset,
    steps=None,
    exporters=[exporter],
    start_delay_secs=10,  # Start evaluating after 10 sec.
    throttle_secs=30  # Evaluate only every 30 sec
)

In [43]:
tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/pilotnetV1/model.ckpt.
INFO:tensorflow:loss = 1.2195287, step = 0
INFO:tensorflow:loss = 1.2195287, mae = 0.85362625
INFO:tensorflow:global_step/sec: 2.561
INFO:tensorflow:loss = 0.52021813, step = 100 (39.048 sec)
INFO:tensorflow:loss = 0.52021813, mae = 0.70833004 (39.048 sec)
INFO:tensorflow:global_step/sec: 2.83833
INFO:tensorflow:loss = 0.25363985, step = 200 (

INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/pilotnetV1/model.ckpt-2978
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/pilotnetV1/export/test_exporter/temp-b'1551653734'/saved_model.pb
INFO:tensorflow:global_step/sec: 0.576491
INFO:tensorflow:loss = 0.019372234, step = 3000 (173.462 sec)
INFO:tensorflow:loss = 0.019372234, mae = 0.18512557 (173.462 sec)
INFO:tensorflow:global_step/sec: 2.87175
INFO:tensorflow:loss = 0.019387271, step = 3100 (34.822 sec)
INFO:tensorflow:loss = 0.019387271, mae = 0.1829902 (34.822 sec)
INFO:tensorflow:global_step/sec: 2.87356
INFO:tensorflow:loss = 0.011181397, step = 3200 (34.800 sec)
INFO:tensorflow:loss = 0.011181397, mae = 0.18019202 (34.800 sec)
INFO:tensorflow:global_step/sec: 2.83773
INFO:tensorflow:loss = 0.014982037, step = 3300 (35.240 sec)
INFO:ten

INFO:tensorflow:loss = 0.007177418, mae = 0.1379938 (37.175 sec)
INFO:tensorflow:global_step/sec: 2.47663
INFO:tensorflow:loss = 0.010056267, step = 6100 (40.376 sec)
INFO:tensorflow:loss = 0.010056267, mae = 0.1371266 (40.376 sec)
INFO:tensorflow:global_step/sec: 2.52704
INFO:tensorflow:loss = 0.008316172, step = 6200 (39.571 sec)
INFO:tensorflow:loss = 0.008316172, mae = 0.13617037 (39.572 sec)
INFO:tensorflow:global_step/sec: 2.64784
INFO:tensorflow:loss = 0.009734729, step = 6300 (37.767 sec)
INFO:tensorflow:loss = 0.009734729, mae = 0.1353182 (37.767 sec)
INFO:tensorflow:global_step/sec: 2.7252
INFO:tensorflow:loss = 0.011791049, step = 6400 (36.694 sec)
INFO:tensorflow:loss = 0.011791049, mae = 0.13469222 (36.694 sec)
INFO:tensorflow:global_step/sec: 2.70664
INFO:tensorflow:loss = 0.008581763, step = 6500 (36.945 sec)
INFO:tensorflow:loss = 0.008581763, mae = 0.13382615 (36.946 sec)
INFO:tensorflow:global_step/sec: 2.85463
INFO:tensorflow:loss = 0.010016886, step = 6600 (35.032 s

INFO:tensorflow:Restoring parameters from /tmp/pilotnetV1/model.ckpt-9113
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-03-23:45:38
INFO:tensorflow:Saving dict for global step 9113: global_step = 9113, loss = 0.006092783, mae/mae = 0.06381908
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9113: /tmp/pilotnetV1/model.ckpt-9113
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/pilotnetV1/model.ckpt-9113
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:S

INFO:tensorflow:loss = 0.007496371, step = 11800 (35.264 sec)
INFO:tensorflow:loss = 0.007496371, mae = 0.10904785 (35.263 sec)
INFO:tensorflow:global_step/sec: 2.84585
INFO:tensorflow:loss = 0.006706059, step = 11900 (35.139 sec)
INFO:tensorflow:loss = 0.006706059, mae = 0.10870628 (35.164 sec)
INFO:tensorflow:global_step/sec: 2.83787
INFO:tensorflow:loss = 0.0042830696, step = 12000 (35.238 sec)
INFO:tensorflow:loss = 0.0042830696, mae = 0.10823637 (35.213 sec)
INFO:tensorflow:global_step/sec: 2.79338
INFO:tensorflow:loss = 0.01523753, step = 12100 (35.799 sec)
INFO:tensorflow:loss = 0.01523753, mae = 0.10818579 (35.799 sec)
INFO:tensorflow:global_step/sec: 2.76035
INFO:tensorflow:loss = 0.004586321, step = 12200 (36.228 sec)
INFO:tensorflow:loss = 0.004586321, mae = 0.10775248 (36.228 sec)
INFO:tensorflow:global_step/sec: 2.75341
INFO:tensorflow:loss = 0.008522853, step = 12300 (36.319 sec)
INFO:tensorflow:loss = 0.008522853, mae = 0.107558206 (36.318 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:global_step/sec: 2.87232
INFO:tensorflow:loss = 0.0067422185, step = 15100 (34.814 sec)
INFO:tensorflow:loss = 0.0067422185, mae = 0.10077001 (34.813 sec)
INFO:tensorflow:global_step/sec: 2.86626
INFO:tensorflow:loss = 0.005160658, step = 15200 (34.890 sec)
INFO:tensorflow:loss = 0.005160658, mae = 0.10049783 (34.891 sec)
INFO:tensorflow:global_step/sec: 2.78156
INFO:tensorflow:loss = 0.004239121, step = 15300 (35.952 sec)
INFO:tensorflow:loss = 0.004239121, mae = 0.1001434 (35.952 sec)
INFO:tensorflow:Saving checkpoints for 15375 into /tmp/pilotnetV1/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-04-00:32:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/pilotnetV1/model.ckpt-15375
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-04-00:35:02
INFO:tensorflow:Saving dict 

({'global_step': 15375, 'loss': 0.0058188234, 'mae/mae': 0.06135876},
 [b'/tmp/pilotnetV1/export/test_exporter/1551659702'])

In [23]:
classifier.export_savedmodel(
    "export/test3_keras_no_bias_2",
    lambda: serving_input_fn()
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Restoring parameters from /tmp/pilotnetV1/model.ckpt-15375
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/test3_keras_no_bias_2/temp-b'1551669038'/saved_model.pb


b'export/test3_keras_no_bias_2/1551669038'